# 1. 라이브러리 불러오기

In [1]:
import pandas as pd

import time
import requests
import warnings
warnings.filterwarnings(action = 'ignore')

from selenium import webdriver
from bs4 import BeautifulSoup

from pyproj import Proj
from pyproj import transform

In [2]:
# WGS84 좌표계
WGS84 = {'proj':'latlong', 'datum':'WGS84', 'ellps':'WGS84'}

# GRS80 좌표계
GRS80 = { 'proj':'tmerc', 'lat_0':'38', 'lon_0':'127', 'k':1, 'x_0':200000, 'y_0':600000, 'ellps':'GRS80', 'units':'m' }

In [3]:
# GRS80 좌표를 WGS84 좌표로 바꿔주는 함수 생성
def grs80_to_wgs84(data):
    return pd.Series(transform(Proj(**GRS80), Proj(**WGS84), data['GRS80_X좌표'], data['GRS80_Y좌표']))

In [26]:
# Open Api로 수집되는 데이터를 저장하기 위한 리스트 객체 생성
czid_list = []
directioncode_list = []
grs80x_list = []
grs80y_list = []
roadgradecode_list = []
roadgradename_list = [] 
routename_list = []
routeno_list = []
shift_list = []
vdscode_list = []
vdsendshift_list = []
vdsid_list = []
vdslength_list = []
vdsname_list = []
vdsstartshift_list = []

In [27]:
for i in range(1, 10):
    url = 'http://data.ex.co.kr/openapi/vdsinfo/vdsList?key=1908225472&type=xml&numOfRows=100&pageNo={}'.format(i)
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'lxml')
    
    czid = soup.find_all('czid')
    directioncode = soup.find_all('directioncode')
    grs80x = soup.find_all('grs80x')
    grs80y = soup.find_all('grs80y')
    roadgradecode = soup.find_all('roadgradecode')
    roadgradename = soup.find_all('roadgradename')
    routename = soup.find_all('routename')
    routeno = soup.find_all('routeno')
    shift = soup.find_all('shift')
    vdscode = soup.find_all('vdscode')
    vdsendshift = soup.find_all('vdsendshift')
    vdsid = soup.find_all('vdsid')
    vdslength = soup.find_all('vdslength')
    vdsname = soup.find_all('vdsname')
    vdsstartshift = soup.find_all('vdsstartshift')
    
    for item in czid:
        czid_list.append(item.text)
    for item in directioncode:
        directioncode_list.append(item.text)
    for item in grs80x:
        grs80x_list.append(item.text)
    for item in grs80y:
        grs80y_list.append(item.text)
    for item in roadgradecode:
        roadgradecode_list.append(item.text)
    for item in roadgradename:
        roadgradename_list.append(item.text)
    for item in routename:
        routename_list.append(item.text)
    for item in routeno:
        routeno_list.append(item.text)
    for item in shift:
        shift_list.append(item.text)
    for item in vdscode:
        vdscode_list.append(item.text)
    for item in vdsendshift:
        vdsendshift_list.append(item.text)
    for item in vdsid:
        vdsid_list.append(item.text)
    for item in vdslength:
        vdslength_list.append(item.text)
    for item in vdsname:
        vdsname_list.append(item.text)
    for item in vdsstartshift:
        vdsstartshift_list.append(item.text)

In [28]:
VDS_data = pd.DataFrame()

VDS_data['VDS_ID'] = vdsid_list
VDS_data['GRS80_X좌표'] = grs80x_list
VDS_data['GRS80_Y좌표'] = grs80y_list
VDS_data['지점이정'] = shift_list
VDS_data['VDS존_시작이정'] = vdsstartshift_list
VDS_data['VDS존_종료이정'] = vdsendshift_list
VDS_data['노선번호'] = routeno_list
VDS_data['도로명'] = routename_list
VDS_data['VDS존_유형구분코드'] = vdscode_list
VDS_data['VDS존_유형구분명'] = vdsname_list
VDS_data['기점종점방향구분코드'] = directioncode_list
VDS_data['VDS존 길이'] = vdslength_list
VDS_data['도로등급구분코드'] = roadgradecode_list
VDS_data['도로등급구분명'] = roadgradename_list
VDS_data['콘존ID'] = czid_list

In [29]:
VDS_data['도로명'].value_counts()

경부선           833
남해선(순천-부산)     58
Name: 도로명, dtype: int64

In [30]:
VDS_경부선_data = VDS_data[VDS_data['도로명'] == '경부선']
VDS_경부선_data.shape

(833, 15)

In [31]:
# 'grs80_to_wgs84' 함수를 사용하여 좌표 변환
VDS_경부선_data[['WGS84_X좌표', 'WGS84_Y좌표']] = VDS_경부선_data.apply(grs80_to_wgs84, axis = 1)

In [33]:
VDS_경부선_data.head(3)

,VDS_ID,GRS80_X좌표,GRS80_Y좌표,지점이정,VDS존_시작이정,VDS존_종료이정,노선번호,도로명,VDS존_유형구분코드,VDS존_유형구분명,기점종점방향구분코드,VDS존 길이,도로등급구분코드,도로등급구분명,콘존ID,WGS84_X좌표,WGS84_Y좌표
0,0010VDE00100,391265.168631,297630.018242,1.40km,0.20km,2.02km,0010,경부선,1,FTMS 본선 VDS,E,1820.00m,101,고속국도,0010CZE010,129.101614,35.256977
1,0010VDE00200,391642.598473,298552.156640,2.40km,2.02km,2.85km,0010,경부선,1,FTMS 본선 VDS,E,830.00m,101,고속국도,0010CZE011,129.105973,35.265211
2,0010VDE00300,391873.613139,299421.826509,3.30km,2.85km,4.01km,0010,경부선,1,FTMS 본선 VDS,E,1160.00m,101,고속국도,0010CZE011,129.108714,35.273000


In [ ]:
VDS_경부선_data.to_csv('경부선 VDS 설치정보.csv', encoding = 'CP949')